# The Eikon API

## Proxy connection

In the previous Episodes we saw how could we obtain data from Eikon using the Python API to connect to its Terminal. We saw it was very easy and straightforward to obtain a constant stream of information or the historic data for a particular financial instrument, using an app key and the API methods.

I spent a lot of nice words praising the ease of connection and the simplicity of the API’s design. The time has come to see the flip of the coin. I will introduce it by telling you a story.

## How does the Python Eikon API get data and how I discovered it

When I started developing applications for my job I thought that and API works always in the same way: you make a request to a remote server via some authentication keys and you get back the response from the server with the information you need. That is what I thought also when connecting to Eikon via my app key. I thought it was my key to authenticate to the Eikon server and get data. 

I was excited the first time I tried my connection so I quickly fired up a Jupyter Notebook to tried out my fresh app key, only to discover with my horror that the eikon proxy was throwing an error:

> ```[MainThread 31936] Eikon Proxy not running or cannot be reached. Please read the documentation on troubleshooting.```

I was a little bit puzzled about the error, and I was thinking that a sort of firewall was preventing me to connect. I tried to see if I had proxies on... but no, everything was ok. So I did the number one thing in the list of the debugging cookbook: Googling the error. 
I found out that there is a developers forum where you can pose questions of any kind regarding developing applications with Eikon and you can find answers to recurrent problems. You can find it [here](https://community.developers.refinitiv.com/tags/eikon.html). 

I also realized that the connection between your Python application and Eikon via the app key serves only to connect to your desktop terminal, that should be up and running and you should be already logged in. 😱

The actual data will be requested by your Terminal and not by the Python application itself. The API is only a bridge between your python script and the Eikon desktop. What happens technically is that the Eikon desktop is running on a port on your Laptop (usually 9000 or 9060), and the Python API will make a request to that port to obtain the data. 

So, in the end, there is no real interaction between you (your script) and a remote server. I was shocked! 'What is the problem!' you will say. As long as you have a connection to a server you will always get your data and you will be able to write your applications. Yes this is true and I can still do it... but I have a developer's mentality and the deployment of your solution is a fundamental step in the process. 

## How to deploy my application that uses Python Eikon API? 

My first question, after having discovered the issue was: "How can I deploy my application? can I do it on the cloud?". Based on what we said, we always need a machine with a Eikon Desktop Terminal up and running. So, deploying the solution on a Linux virtual machine on the cloud is impossible, we need a Windows machine. Moreover, we need a machine with the Eikon Desktop installed and a valid account on it. Disappointing! 😡

I had the confimation making a search in the developers community forum and I found out that I was not alone. Someone before me asked the same (or similar) question:

<img src="images/Eikon_Proxy_Question.png">

and one of the answers:

<img src="images/Eikon_Proxy_Answer.png">

you can follow the trend [here](https://community.developers.refinitiv.com/questions/48321/how-can-i-use-the-eikon-data-api-without-signing-i.html). Leter on, I started thinking about what would be the best solution to deploy my applications, and the only two that came in my mind were: 
1. A physical server you can connect via VNC or similar. 
2. A virtual Desktop in Microsoft Azure Client.

If you have a better solution please let me know!

After 3 years of work in Finance I finally realize why this diabolic connection method works in this way. Data providers firms like Bloomberg or Reuters try to exploit revenues not only from the single user, but for the single machine accessing the service. Every machine should pay a license fee, so that you cannot access the API from wherever you want, i.e. external hosts like cloud servers. 

This is particularly true with providers like Bloomberg, where the level of control over the IPs reaches a paranoic level. The Client usually kicks you out if the desktop goes aspleep and obliges you to re-connect, just to be sure to check your location every-day. Even with a particular subscription called 'everywhere' you recerive a device for fingerprint authentication as a next level of security. Always to be sure that is really you, and that you didn't lend your terminal to some other guy.

Eventually also your Eikon Desktop will kick you out. In my case it happens during week-ends and unfortunately I have to deal with it and find a workaround for that. 

## How to check if the Eikon Desktop is up and running

In order to deploy a robust application we should be able to check if the Desktop app is up and running. We may need this check, in particular if there is a scheduled job that runs every X time and, in example, feeds a database with some instrument past data. We don't want to trigger a job that crashes already in the first lines if the Application is not running.

To do this we will use some interesting python libraries that every developer should be familiar with: os, subprocess and requests and also a simple method from the API:

## ```get_port_number()```

Returns the port number used to communicate with the Eikon Data API Proxy.

### Structure

```get_port_number()```

The method returns ```None``` if the Eikon terminal is not running. 

The port is either 9060 or 9000. If we want to control if the proxy is running we have two possible methods: use ```get_port_number``` and check if the result is not ```None``` or send an http request to the 9060 port. 

I personally prefer using the method from the API, because it will avoid the hard-coding of the port number in the request, but I will still show you the two methods:

### #1 Use the API method

In [2]:
import eikon

# connect to the eikon api
eikon.set_app_key('07d8b517d50c495d9784b0bcad37c3f5f879e27f')

# get port number to communicate
# with the Eikon Terminal
port = eikon.get_port_number()

if port: 
    print(f'The Eikon proxy is up and running on port {port}')
else:
    print('It seems your Eikon Terminal is not running')
    # take action here

The Eikon proxy is up and running on port 9060


### #2 making an http request 

We will use the python library ```requests``` and send an http request to localhost 9060 and route api/status and look at the response.

In [1]:
import requests

# check if eikon application is opened
try:
    # send a request to port 9060
    # returns a status 'ST_PROXY_READY' if eikon application is opened
    # throws a timeout error if the application is closed
    request = requests.get('http://localhost:9060/api/status')
    response = request.json()

    if response['statusCode'] == 'ST_PROXY_READY': 
        print('The Eikon proxy is up and running')
except:
    print('It seems your Eikon Terminal is not running')
    # take action here

The Eikon proxy is up and running


In this case, the request throws a timeout error if the application is not running and we need to wrap our request into a try-except statement.

## What to do if the Terminal is not running

Notice that I left a #take action here comment if the Terminal is not running. In this case we want to do something meaningful. This can be a simple notification telling the user to switch on the application, manually. But we are python samurais ⛩, our motto is: 

> _no mercy for manual entry_ 

and we are here to try to automatize the most our application, so we will make our machine open the terminal for us. The executable to launch the terminal is usually in ```C:\Program Files (x86)\Thomson Reuters\Eikon\Eikon.exe```. Python can lanch windows shell commands via the ```subprocess``` module or launch windows executable using the module ```os```. Either of the two following commands will launch the Terminal

In [1]:
# using os startfile method to launch the Eikon Terminal
import os
os.startfile("C:\Program Files (x86)\Thomson Reuters\Eikon\Eikon.exe")

# using the subprocess check_call method to launch the Eikon Terminal
import subprocess
subprocess.check_call([r"C:\Program Files (86)\Thomson Reuters\Eikon\Eikon.exe"])

In [4]:
import subprocess
subprocess.check_call([r"C:\Program Files (86)\Thomson Reuters\Eikon\Eikon.exe"])

FileNotFoundError: [WinError 2] Impossibile trovare il file specificato

I used the ```strftime``` method to convert from datetime to convert from ```datetime``` to ```string```. We can now try to retrieve the actual data using those dates. Let's say we want Microsoft and Apple candles (open, close, high, low). The code will be the following:

In [4]:
import eikon

# connect to the eikon api
eikon.set_app_key('07d8b517d50c495d9784b0bcad37c3f5f879e27f')
port = eikon.get_port_number()

print(port)

9060


Two things to notice:
1. it is a multi-index dataframe 
2. the week-end dates are not taken into account

you can access the single dataframes in this way

In [8]:
df["MSFT.O"]

Field,OPEN,CLOSE,HIGH,LOW
Date,,,,
2022-02-10,304.04,302.38,309.12,300.7
2022-02-11,303.19,295.04,304.29,294.22
2022-02-14,293.77,295.0,296.76,291.35
2022-02-15,300.008,300.47,300.8,297.02
2022-02-16,298.365,294.475,298.66,293.68


Some other parameters may be interesting. For the interval, you can go until ```tick```

In [14]:
# get candle for a couple of instruments
df = eikon.get_timeseries(["MSFT.O"], 
                        start_date = start_datetime,
                        end_date = end_datetime, 
                        interval="tick")
df

MSFT.O,VALUE,VOLUME
Date,,
2022-02-16 15:10:35.721,294.43,100
2022-02-16 15:10:35.721,294.41,100
2022-02-16 15:10:35.721,294.4,1
2022-02-16 15:10:35.721,294.42,18
2022-02-16 15:10:35.721,294.42,15
...,...,...
2022-02-16 15:51:12.656,294.9,6
2022-02-16 15:51:12.656,294.9,100
2022-02-16 15:51:12.663,294.895,20


For this only the VALUE and VOLUME fields are present. If you try to query for other fields you will have a DataFrame full of NAs. As you can see, you can easily rump it up to a DataFrame with 50000 rows. The query took 2.5 seconds to run!

You can start playing a bit with the parameters and see what changes, just to have a feeling of how it works to retrieve the data. I promise it will be fun! 🤪

For the moment we saw how to retrieve a stream of data and now you also know how to retrieve past data. Well, I think you are super ready to start building your applications. Once you have the data at hand it's already half of the job. Try to calculate something with the DataFrame you have, like total return or volatility, you will see it's very easy and satisfactory. 

But we're not finished yet. I would like to discuss a couple of more advanced features that you can build in Python, for example how to create a Python Class that will do the job of connecting and retrieving data with yet less effort. What to say: stay with the fellowship! 🧙‍♂️ 